In [2]:
# notebook to confirm that modifying only part of the A matrix/b vector when changing m0 works.
# extent of time difference between the two is unclear.
import sys
import os
sys.path.append(os.path.relpath('../'))
import numpy as np
from multi_condensed import Problem
import time

In [3]:
h = 100
d = [29, 7, 4]
a = [3, 5, 10]
heaving = [0, 1, 1]
NMK = [100, 100, 100, 100]

m0s = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
rho = 1023

def modify_and_solve(prob, a_matrix, b_vector, m0):
  prob.change_m0(m0)
  a_matrix = prob.a_matrix_from_old(a_matrix)
  b_vector = prob.b_vector_from_old(b_vector)
  x = prob.get_unknown_coeffs(a_matrix, b_vector)
  am, dp = prob.hydro_coeffs(x, "nondimensional")
  return am, dp

def create_and_solve(m0):
  prob = Problem(h, d, a, heaving, NMK, m0, rho)
  a_matrix = prob.a_matrix()
  b_vector = prob.b_vector()
  x = prob.get_unknown_coeffs(a_matrix, b_vector)
  am, dp = prob.hydro_coeffs(x, "nondimensional")
  return am, dp

In [3]:
am_modified = []
dp_modified = []
am_recreate = []
dp_recreate = []

start = time.perf_counter()
prob = Problem(h, d, a, heaving, NMK, 1, rho)
a0 = prob.a_matrix()
b0 = prob.b_vector()
end = time.perf_counter()
tsingle = end - start

start = time.perf_counter()
for m0 in m0s:
  am, dp = modify_and_solve(prob, a0, b0, m0)
  am_modified.append(am)
  dp_modified.append(dp)
end = time.perf_counter()
tmod = end - start

start = time.perf_counter()
for m0 in m0s:
  am, dp = create_and_solve(m0)
  am_recreate.append(am)
  dp_recreate.append(dp)
end = time.perf_counter()
tcreate = end - start

print(tsingle, tmod, tcreate)

for i in range(len(am_modified)):
  print(am_modified[i] - am_recreate[i])

for i in range(len(dp_modified)):
  print(dp_modified[i] - dp_recreate[i])


0.5318210260011256 9.564884053019341 10.276806697016582
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [16]:
# Breakdown of how long each step takes.
start = time.perf_counter()
prob = Problem(h, d, a, heaving, NMK, 1, rho)
end = time.perf_counter()
print("Initialize:", end - start)

start = time.perf_counter()
a0 = prob.a_matrix()
end = time.perf_counter()
print("Full A-matrix:", end - start)

start = time.perf_counter()
b0 = prob.b_vector()
end = time.perf_counter()
print("Full b-vector", end - start)

start = time.perf_counter()
x = prob.get_unknown_coeffs(a0, b0)
end = time.perf_counter()
print("Matrix Solve:", end - start)

start = time.perf_counter()
am, dp = prob.hydro_coeffs(x, "nondimensional")
end = time.perf_counter()
print("Hydro Coeff Computation:", end - start)

start = time.perf_counter()
prob.change_m0(3)
end = time.perf_counter()
print("Changing m0:", end - start)

start = time.perf_counter()
a_matrix = prob.a_matrix_from_old(a0)
end = time.perf_counter()
print("Changing A-matrix:", end - start)

start = time.perf_counter()
b_vector = prob.b_vector_from_old(b0)
end = time.perf_counter()
print("Changing b-vector:", end - start)

Initialize: 0.4658776489959564
Full A-matrix: 0.17097103298874572
Full b-vector 0.0020133999933023006
Matrix Solve: 0.017734749999362975
Hydro Coeff Computation: 0.007879554003011435
Changing m0: 0.4006503580021672
Changing A-matrix: 0.08661461900919676
Changing b-vector: 0.0007198549865279347
